# Configuration

> daisyUI configuration management for FastHTML projects

In [ ]:
#| default_exp core.config

In [ ]:
#| export
from typing import List, Optional, Dict, Union, Any, Literal
from dataclasses import dataclass, field
from enum import Enum
from pathlib import Path
import json

from cjm_fasthtml_daisyui.core.types import (
    DaisyUITheme, 
    ExcludeFeature,
    ColorSchemeType,
    BinaryType,
    HTMLAttrs
)

## daisyUI Themes

daisyUI comes with many built-in themes. This module provides type-safe access to all available themes and configuration options.

In [ ]:
#| export
@dataclass
class ThemeConfig:
    """Configuration for a single theme"""
    name: Union[DaisyUITheme, str]
    is_default: bool = False
    is_prefers_dark: bool = False
    
    def to_string(
        self
    ) -> str:  # daisyUI config string
        """Convert to daisyUI config string format"""
        theme_name = self.name.value if isinstance(self.name, DaisyUITheme) else self.name
        
        flags = []
        if self.is_default:
            flags.append("--default")
        if self.is_prefers_dark:
            flags.append("--prefersdark")
        
        if flags:
            return f"{theme_name} {' '.join(flags)}"
        return theme_name

In [ ]:
#| export
@dataclass
class DaisyUIConfig:
    """
    Complete daisyUI configuration for FastHTML projects
    """
    # List of themes to include (can be theme names or ThemeConfig objects)
    themes: List[Union[DaisyUITheme, str, ThemeConfig]] = field(default_factory=lambda: [
        ThemeConfig(DaisyUITheme.LIGHT, is_default=True),
        ThemeConfig(DaisyUITheme.DARK, is_prefers_dark=True)
    ])
    root: str = ":root" # CSS selector for root element
    include: List[str] = field(default_factory=list) # Components to explicitly include (empty means all)
    exclude: List[Union[ExcludeFeature, str]] = field(default_factory=list) # Components to exclude
    prefix: str = "" # Prefix for all daisyUI classes
    logs: bool = True # Enable/disable console logs
    
    def add_theme(
        self, 
        theme: Union[DaisyUITheme, str],
        is_default: bool = False,  # If True, sets this theme as the default theme
        is_prefers_dark: bool = False  # If True, sets this theme as the preferred dark theme
    ) -> "DaisyUIConfig":  # Returns self for method chaining
        """Add a theme to the configuration"""
        theme_config = ThemeConfig(theme, is_default, is_prefers_dark)
        
        # If setting as default, unset other defaults
        if is_default:
            for t in self.themes:
                if isinstance(t, ThemeConfig):
                    t.is_default = False
        
        # If setting as prefers dark, unset other prefers dark
        if is_prefers_dark:
            for t in self.themes:
                if isinstance(t, ThemeConfig):
                    t.is_prefers_dark = False
        
        self.themes.append(theme_config)
        return self
    
    def exclude_feature(
        self,
        feature: Union[ExcludeFeature, str]  # Feature to exclude from daisyUI
    ) -> "DaisyUIConfig":  # Returns self for method chaining
        """Exclude a feature from daisyUI"""
        if feature not in self.exclude:
            self.exclude.append(feature)
        return self
    
    def set_prefix(
        self,
        prefix: str  # Prefix to add to all daisyUI class names
    ) -> "DaisyUIConfig":  # Returns self for method chaining
        """Set a prefix for all daisyUI classes"""
        self.prefix = prefix
        return self
    
    def _format_themes(
        self
    ) -> str:  # Formatted theme string for CSS output
        """Format themes for CSS output"""
        theme_strings = []
        
        for theme in self.themes:
            if isinstance(theme, ThemeConfig):
                theme_strings.append(theme.to_string())
            elif isinstance(theme, DaisyUITheme):
                theme_strings.append(theme.value)
            else:
                theme_strings.append(str(theme))
        
        return ", ".join(theme_strings)
    
    def _format_exclude(
        self
    ) -> str:  # Formatted exclude string for CSS output
        """Format exclude list for CSS output"""
        exclude_strings = []
        
        for item in self.exclude:
            if isinstance(item, ExcludeFeature):
                exclude_strings.append(item.value)
            else:
                exclude_strings.append(str(item))
        
        return ", ".join(exclude_strings)
    
    def to_css(
        self
    ) -> str:  # Complete CSS configuration string for daisyUI
        """Generate CSS configuration string"""
        lines = ['@import "tailwindcss";']
        
        # Check if we need any configuration
        has_config = (
            self.themes != [
                ThemeConfig(DaisyUITheme.LIGHT, is_default=True),
                ThemeConfig(DaisyUITheme.DARK, is_prefers_dark=True)
            ] or
            self.root != ":root" or
            self.include or
            self.exclude or
            self.prefix or
            not self.logs
        )
        
        if not has_config:
            lines.append('@plugin "daisyui";')
        else:
            lines.append('@plugin "daisyui" {')
            
            # Add themes
            lines.append(f'  themes: {self._format_themes()};')
            
            # Add root if not default
            if self.root != ":root":
                lines.append(f'  root: "{self.root}";')
            
            # Add include if specified
            if self.include:
                include_str = ", ".join(self.include)
                lines.append(f'  include: {include_str};')
            
            # Add exclude if specified
            if self.exclude:
                lines.append(f'  exclude: {self._format_exclude()};')
            
            # Add prefix if specified
            if self.prefix:
                lines.append(f'  prefix: {self.prefix};')
            
            # Add logs setting if false
            if not self.logs:
                lines.append('  logs: false;')
            
            lines.append('}')
        
        return '\n'.join(lines)

## Custom Theme Support

Support for creating custom daisyUI themes:

In [ ]:
#| export
@dataclass
class ColorScheme:
    """Color scheme for a custom theme"""
    # Base colors
    base_100: str = "oklch(98% 0.02 240)"
    base_200: str = "oklch(95% 0.03 240)"
    base_300: str = "oklch(92% 0.04 240)"
    base_content: str = "oklch(20% 0.05 240)"
    
    # Brand colors
    primary: str = "oklch(55% 0.3 240)"
    primary_content: str = "oklch(98% 0.01 240)"
    secondary: str = "oklch(70% 0.25 200)"
    secondary_content: str = "oklch(98% 0.01 200)"
    accent: str = "oklch(65% 0.25 160)"
    accent_content: str = "oklch(98% 0.01 160)"
    
    # Semantic colors
    neutral: str = "oklch(50% 0.05 240)"
    neutral_content: str = "oklch(98% 0.01 240)"
    info: str = "oklch(70% 0.2 220)"
    info_content: str = "oklch(98% 0.01 220)"
    success: str = "oklch(65% 0.25 140)"
    success_content: str = "oklch(98% 0.01 140)"
    warning: str = "oklch(80% 0.25 80)"
    warning_content: str = "oklch(20% 0.05 80)"
    error: str = "oklch(65% 0.3 30)"
    error_content: str = "oklch(98% 0.01 30)"
    
    def to_css_vars(
        self
    ) -> HTMLAttrs:  # Dictionary of CSS variable names to color values
        """Convert to CSS variable format"""
        return {
            "--color-base-100": self.base_100,
            "--color-base-200": self.base_200,
            "--color-base-300": self.base_300,
            "--color-base-content": self.base_content,
            "--color-primary": self.primary,
            "--color-primary-content": self.primary_content,
            "--color-secondary": self.secondary,
            "--color-secondary-content": self.secondary_content,
            "--color-accent": self.accent,
            "--color-accent-content": self.accent_content,
            "--color-neutral": self.neutral,
            "--color-neutral-content": self.neutral_content,
            "--color-info": self.info,
            "--color-info-content": self.info_content,
            "--color-success": self.success,
            "--color-success-content": self.success_content,
            "--color-warning": self.warning,
            "--color-warning-content": self.warning_content,
            "--color-error": self.error,
            "--color-error-content": self.error_content,
        }

In [ ]:
#| export
@dataclass
class ThemeDesignTokens:
    """Design tokens for theme customization"""
    # Border radius
    radius_selector: str = "1rem"  # For checkbox, toggle, badge
    radius_field: str = "0.25rem"  # For button, input, select, tab
    radius_box: str = "0.5rem"     # For card, modal, alert
    
    # Component sizing
    size_selector: str = "0.25rem"  # Base size for selectors
    size_field: str = "0.25rem"     # Base size for fields
    
    # Visual properties
    border: str = "1px"             # Border width
    depth: BinaryType = 1           # Shadow and 3D effect (0 or 1)
    noise: BinaryType = 0           # Noise texture effect (0 or 1)
    
    def to_css_vars(
        self
    ) -> HTMLAttrs:  # Dictionary of CSS variable names to design token values
        """Convert to CSS variable format"""
        return {
            "--radius-selector": self.radius_selector,
            "--radius-field": self.radius_field,
            "--radius-box": self.radius_box,
            "--size-selector": self.size_selector,
            "--size-field": self.size_field,
            "--border": self.border,
            "--depth": str(self.depth),
            "--noise": str(self.noise),
        }

In [ ]:
#| export
@dataclass
class CustomTheme:
    """Complete custom theme definition"""
    name: str
    is_default: bool = False
    is_prefers_dark: bool = False
    color_scheme: ColorSchemeType = "light"
    colors: ColorScheme = field(default_factory=ColorScheme)
    tokens: ThemeDesignTokens = field(default_factory=ThemeDesignTokens)
    
    def to_css(
        self
    ) -> str:  # CSS plugin configuration for the custom theme
        """Generate CSS for the custom theme"""
        lines = ['@plugin "daisyui/theme" {']
        lines.append(f'  name: "{self.name}";')
        
        if self.is_default:
            lines.append('  default: true;')
        
        if self.is_prefers_dark:
            lines.append('  prefersdark: true;')
        
        lines.append(f'  color-scheme: {self.color_scheme};')
        lines.append('')
        
        # Add color variables
        for var_name, value in self.colors.to_css_vars().items():
            lines.append(f'  {var_name}: {value};')
        
        lines.append('')
        
        # Add design token variables
        for var_name, value in self.tokens.to_css_vars().items():
            lines.append(f'  {var_name}: {value};')
        
        lines.append('}')
        
        return '\n'.join(lines)

## Usage Examples

Here are common patterns for configuring daisyUI in FastHTML projects:

### Basic Configuration

In [ ]:
# Default configuration
config = DaisyUIConfig()
print("Default config:")
print(config.to_css())

Default config:
@import "tailwindcss";
@plugin "daisyui";


### Theme Selection

In [ ]:
# Configure specific themes
config = DaisyUIConfig(themes=[])
config.add_theme(DaisyUITheme.CUPCAKE, is_default=True)
config.add_theme(DaisyUITheme.DRACULA, is_prefers_dark=True)
config.add_theme(DaisyUITheme.CYBERPUNK)

print("Custom theme selection:")
print(config.to_css())

Custom theme selection:
@import "tailwindcss";
@plugin "daisyui" {
  themes: cupcake --default, dracula --prefersdark, cyberpunk;
}


### Feature Exclusion

In [ ]:
# Exclude specific features
config = DaisyUIConfig()
config.exclude_feature(ExcludeFeature.ROOT_SCROLL_GUTTER)
config.exclude_feature(ExcludeFeature.CHECKBOX)
config.set_prefix("daisy-")

print("Exclusion and prefix config:")
print(config.to_css())

Exclusion and prefix config:
@import "tailwindcss";
@plugin "daisyui" {
  themes: light --default, dark --prefersdark;
  exclude: rootscrollgutter, checkbox;
  prefix: daisy-;
}


### Custom Theme Creation

In [ ]:
# Create a custom theme
custom_colors = ColorScheme(
    primary="oklch(60% 0.3 250)",
    primary_content="oklch(95% 0.02 250)",
    secondary="oklch(75% 0.2 180)",
    secondary_content="oklch(95% 0.02 180)"
)

custom_theme = CustomTheme(
    name="mytheme",
    is_default=True,
    color_scheme="light",
    colors=custom_colors,
    tokens=ThemeDesignTokens(
        radius_field="0.5rem",
        radius_box="1rem",
        depth=0  # Flat design
    )
)

print("Custom theme:")
print(custom_theme.to_css())

Custom theme:
@plugin "daisyui/theme" {
  name: "mytheme";
  default: true;
  color-scheme: light;

  --color-base-100: oklch(98% 0.02 240);
  --color-base-200: oklch(95% 0.03 240);
  --color-base-300: oklch(92% 0.04 240);
  --color-base-content: oklch(20% 0.05 240);
  --color-primary: oklch(60% 0.3 250);
  --color-primary-content: oklch(95% 0.02 250);
  --color-secondary: oklch(75% 0.2 180);
  --color-secondary-content: oklch(95% 0.02 180);
  --color-accent: oklch(65% 0.25 160);
  --color-accent-content: oklch(98% 0.01 160);
  --color-neutral: oklch(50% 0.05 240);
  --color-neutral-content: oklch(98% 0.01 240);
  --color-info: oklch(70% 0.2 220);
  --color-info-content: oklch(98% 0.01 220);
  --color-success: oklch(65% 0.25 140);
  --color-success-content: oklch(98% 0.01 140);
  --color-warning: oklch(80% 0.25 80);
  --color-warning-content: oklch(20% 0.05 80);
  --color-error: oklch(65% 0.3 30);
  --color-error-content: oklch(98% 0.01 30);

  --radius-selector: 1rem;
  --radius-field

## Configuration Persistence

Save and load configurations:

In [ ]:
#| export
class ConfigManager:
    """Manage daisyUI configuration persistence"""
    
    @staticmethod
    def save_to_file(
        config: DaisyUIConfig,  # Configuration object to save
        path: Union[str, Path]  # File path where the CSS will be saved
    ) -> None:
        """Save configuration to a CSS file"""
        path = Path(path)
        path.write_text(config.to_css())
    
    @staticmethod
    def save_custom_theme(
        theme: CustomTheme,  # Custom theme object to save
        path: Union[str, Path]  # File path where the CSS will be saved
    ) -> None:
        """Save custom theme to a CSS file"""
        path = Path(path)
        path.write_text(theme.to_css())
    
    @staticmethod
    def combine_with_custom_themes(
        config: DaisyUIConfig,  # Base configuration
        custom_themes: List[CustomTheme]  # List of custom themes to include
    ) -> str:  # Combined CSS string with config and custom themes
        """Combine configuration with custom themes"""
        lines = [config.to_css()]
        
        for theme in custom_themes:
            lines.append('')
            lines.append(theme.to_css())
        
        return '\n'.join(lines)

## Best Practices

1. **Theme Selection**: Choose themes that match your brand and provide good accessibility
2. **Performance**: Exclude unused components to reduce CSS size
3. **Prefixing**: Use prefixes when integrating with existing CSS frameworks
4. **Custom Themes**: Use OKLCH colors for better color consistency across devices
5. **Dark Mode**: Always provide a `prefers-dark` theme for better user experience

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()